In [1]:
import os
#set directories
# ASAP_DIR = r'/home/levi/asap'
SILKROAD_DIR = r'silkroad'
VICE_DIR = r'vice_city'
DARKFOX_DIR = r'darkfox'


def get_names(DIR):
    '''retrieve vendor names from html file in directories'''
    
    files = os.listdir(DIR)
    vendor_names = [i[:-5] for i in files]
    
    return vendor_names
    
# asap_vendors = get_names(ASAP_DIR)
silkroad_vendors = get_names(SILKROAD_DIR)
vice_vendors = get_names(VICE_DIR)
darkfox_vendors = get_names(DARKFOX_DIR)

all_vendors = darkfox_vendors + silkroad_vendors + vice_vendors
#  + asap_vendors
all_vendors = list(dict.fromkeys(all_vendors)) #remove duplicates
#list(set(vice).intersection(asap))

In [2]:
#df.loc[df['name'] == 'DopeNL', 'products'] = 10

#list of common marketplaces
list_of_markets = ['apollon', 'white house', 'vice city', 'torrez', 'empire', 'silkroad', 'darkfox','asap',
                   'berlusconi', 'cartel','monopoly','icarus','pax romana', 'cryptonia', 'nightmare', 
                   'invictus', 'world market', 'traderoute', 'wall street']

In [3]:
###SCRAPE DATA FROM DARKFOX####
from bs4 import BeautifulSoup as bs


darkfox_data=[] #empty list which will contain our data


for i in range(len(os.listdir(DARKFOX_DIR))): #for each html file
    
    darkfox_df = {} #empty dictionary which will contain user data
    
    categories = []
    completed_orders = [0]
    ships_from = []
    active_markets = []
    
    
    name = os.listdir(DARKFOX_DIR)[i][:-5] #vendor name
      
    
    with open(DARKFOX_DIR+'/' + os.listdir(DARKFOX_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower(): #lowercase text
            active_markets.append(market)
    
        
    #find product categories    
    for line in list(txt):
        if 'Category' in line:
            categories.append(line[10:])
        if 'Ships from' in line:
            ships_from.append(line[12:])
            
            
    #remove duplicates  
    categories = list(dict.fromkeys(categories))
    ships_from = list(dict.fromkeys(ships_from))
    active_markets = list(dict.fromkeys(active_markets))
    
    
    #find number of completed orders
    for idx, line in enumerate(list(txt)): 
            if 'Completed orders:' in line:
                orders = txt[idx+1]
                completed_orders.append(orders)
    
   
    #add name, categories,completed orders, location and known markets
    darkfox_df['name'] = name
    darkfox_df['products'] = categories
    darkfox_df['completed_orders'] = int(completed_orders[-1]) #there are two numbers so only add the last one
    darkfox_df['ships_from'] = ships_from
    #darkfox_df['active_markets'] = active_markets
    
    darkfox_data.append(darkfox_df) #add user to dataframe
        
        
#         if categories != []:
#             df.loc[df['name'] == name, 'products'] = categories
        

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 77712: character maps to <undefined>

In [65]:
darkfox_data = pd.DataFrame(data= darkfox_data) #transform to dataframe
darkfox_data.to_csv('darkfox.csv')
darkfox_data.sort_values('completed_orders', ascending = False)


,name,products,completed_orders,ships_from
56,Schneider,[Cocaine],1378,[Germany]
761,egoods,"[Data & Scans, Personal Info, Accounts & Drops]",1031,[]
245,yoshisteam,"[Security, Fraud, Dumps, Accounts & Drops, Acc...",710,[]
607,Onepiece,"[Fraud, Cards & CVV]",602,[]
133,DrunkNinja,"[Other, Fraud, Malware, Crypters, Hacking, Bot...",583,[]
...,...,...,...,...
274,thcwizzard,[],0,[]
275,TrustMatters,[],0,[]
77,ikumnyilite,"[Cards & CVV, Databases]",0,[]
463,mushmush,[],0,[]


In [73]:
# with open(ASAP_DIR+'/' + os.listdir(ASAP_DIR)[4]) as f:
    
#     soup = bs(f, 'html.parser') #read using beautiful soup
    
#     txt=soup.find_all(text=True)

# name = os.listdir(ASAP_DIR)[4][:-5]

# for line in list(txt):
#     if 'Username' in line:
        


                                        Username:
                                    


In [108]:
###ASAP SCRAPER###

asap_data=[] #empty list which will contain our data

for i in range(len(os.listdir(ASAP_DIR))): #for each html file
    
    asap_df = {} #empty dictionary which will contain user data
    
    active_markets = []
    
    name = os.listdir(ASAP_DIR)[i][:-5] #vendor name
      
    
    with open(ASAP_DIR+'/' + os.listdir(ASAP_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower():
            active_markets.append(market)
    
    #remove duplicates   
    active_markets = list(dict.fromkeys(active_markets))
       
    #add name, categories,completed orders, location and known markets
    asap_df['name'] = name
    asap_df['active_markets'] = active_markets
    
    asap_data.append(asap_df) #add user to dataframe
        
        
#         if categories != []:
#             df.loc[df['name'] == nam

In [109]:
asap_data = pd.DataFrame(data=asap_data)
asap_data.to_csv('asap_data.csv')
asap_data.head()

,name,active_markets
0,pharmatec,[asap]
1,ICEGOD,"[white house, torrez, empire, asap, wall street]"
2,danielvitor61,"[torrez, asap]"
3,QualityFakeIDs,"[white house, empire, asap]"
4,mysteryland,"[apollon, torrez, empire, asap, invictus, worl..."


In [110]:
###VICE_CITY SCRAPER###

vice_data=[] #empty list which will contain our data

for i in range(len(os.listdir(VICE_DIR))): #for each html file
    
    vice_df = {} #empty dictionary which will contain user data
    
    active_markets = []
    
    name = os.listdir(VICE_DIR)[i][:-5] #vendor name
      
    
    with open(VICE_DIR+'/' + os.listdir(VICE_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower():
            active_markets.append(market)
    
    #remove duplicates   
    active_markets = list(dict.fromkeys(active_markets))
       
    #add name, categories,completed orders, location and known markets
    vice_df['name'] = name
    vice_df['active_markets'] = active_markets
    
    vice_data.append(vice_df) #add user to dataframe

In [111]:
vice_data = pd.DataFrame(data=vice_data)
vice_data.to_csv('vice_data.csv')
vice_data.head(15)

,name,active_markets
0,fullzipp,[vice city]
1,k1to,[vice city]
2,mysteryland,[vice city]
3,CherryUK,[vice city]
4,Venture,[vice city]
5,mythbusters,[vice city]
6,shroomery,"[vice city, asap]"
7,TheWizzardNL,[vice city]
8,Crypto2Psychs,[vice city]
9,superhero,[vice city]


In [120]:
###SCRAPE DATA FROM SILKROAD####
from bs4 import BeautifulSoup as bs

silkroad_data=[] #empty list which will contain our data

for i in range(len(os.listdir(SILKROAD_DIR))): #for each html file
    
    silkroad_df = {} #empty dictionary which will contain user data
    
    categories = []
    ships_from = []
    
    name = os.listdir(SILKROAD_DIR)[i][:-5] #vendor name
      
    
    with open(SILKROAD_DIR+'/' + os.listdir(SILKROAD_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
        
    #find product categories    
    for idx,line in enumerate(list(txt)):
        if 'Category' in line:
            category = txt[idx+1] #get text after 'category:'
            categories.append(category)
            
        if 'Ships From' in line:
            ships = txt[idx+1]
            if ships != '\n':
                ships_from.append(ships)
            
            
    #remove duplicates  
    categories = list(dict.fromkeys(categories))
    ships_from = list(dict.fromkeys(ships_from))
    
    
    #add name, categories,completed orders, location and known markets
    silkroad_df['name'] = name
    silkroad_df['products'] = categories
    silkroad_df['ships_from'] = ships_from
    
    silkroad_data.append(silkroad_df) #add user to dataframe
    
silk_data = pd.DataFrame(data=silkroad_data)
silk_data.to_csv('silk_data.csv')
silk_data.head(15)

,name,products,ships_from
0,glasshouse,"[Weed, Concentrates, eBooks, PayPal, Edibles, ...","[United States, Not Specified]"
1,mysteryland,"[Speed, Ecstasy]",[Germany]
2,AmsterdamLiquidXno1,"[LSD, 2C, Ketamine, Oxycodon, Ecstasy]",[Netherlands]
3,docspro,"[Scans, Cocaine, Carded Items, Xanax, Counterf...",[Ukraine]
4,LauraSilver,[Buds & Flowers],[Germany]
5,GRUENEWALDSCAVE,"[Ecstasy, Cocaine]",[United Kingdom]
6,alanmarret,"[Accounts, Banks, CC/CVV, Counterfeits, Carded...",[United States]
7,EmergencyB,[LSD],[Spain]
8,doneycc,[Ships From: ],[United States of America]
9,meds2buy,"[Oxycodon, Ecstasy]",[United States of America]


In [173]:
# build dataframe containing which markets the user is active
import pandas as pd
import numpy as np


test = []
for i in all_vendors:
    
    data = {}
    data['name'] = i
    data['markets'] = []
    data['ships_from'] = []
    data['products'] = []
    
    
    if i in silkroad_vendors:
        vendor_data = silk_data.loc[silk_data.name == i] #find same vendor in df
        shipping = vendor_data['ships_from'].iloc[0] #get shipping
        products = vendor_data['products'].iloc[0]
        
        
        data['markets'].append('silkroad')
        data['ships_from'].extend(shipping)
        data['products'].extend(products)
        
    if i in vice_vendors:
        vendor_data = vice_data.loc[vice_data.name == i] #find same vendor in df
        vendor_markets = vendor_data.active_markets.iloc[0] #get active markets
        
        data['markets'].extend(vendor_markets)

    if i in darkfox_vendors:
        vendor_data = darkfox_data.loc[darkfox_data.name == i] #find same vendor in df
        shipping = vendor_data['ships_from'].iloc[0] #get shipping
        products = vendor_data['products'].iloc[0] #add products
        
        
        data['markets'].append('darkfox')#add market
        data['ships_from'].extend(shipping) #add shipping
        data['products'].extend(products)
        
    if i in asap_vendors:
        vendor_data = asap_data.loc[asap_data.name == i] #find same vendor in df
        vendor_markets = asap_data.active_markets.iloc[0] #get active markets
        
        data['markets'].extend(vendor_markets)
        
    
    data['markets']=list(dict.fromkeys(data['markets'])) #remove duplicates
    data['ships_from']=list(dict.fromkeys(data['ships_from']))
    data['products']=list(dict.fromkeys(data['products']))
    
    data['n_markets'] = len(data['markets'])
    
    test.append(data)

df = pd.DataFrame(data=test)
df.head()

,name,markets,ships_from,products,n_markets
0,Medicamentum,[darkfox],[United Kingdom],[Benzos],1
1,deemzbeemz,[darkfox],[United States],[DMT],1
2,OrganicUKSoil,[darkfox],[United Kingdom],[Weed],1
3,pharmatec,"[darkfox, asap]",[],[],2
4,chapowlini,[darkfox],[Netherlands],"[Cocaine, XTC Pills]",1


In [172]:
df.to_csv('all_users.csv')